This example mirrors [shiny.volumes.html](https://niivue.com/demos/features/shiny.volumes.html).

In [ ]:
import ipywidgets as widgets
from IPython.display import display

import ipyniivue
from ipyniivue import DragMode, ShowRender, SliceType

base_url = "https://niivue.com/demos"
files = [
    "images/mni152.nii.gz",
    "images/spmMotor.nii.gz",
    "images/connectome.jcon",
    "images/dpsv.trx",
]
ipyniivue.download_dataset(api_url=base_url, dest_folder=".", files=files)

ipyniivue.download_dataset(
    api_url="https://niivue.com/demos/matcaps",
    dest_folder="./matcaps",
    files=[
        "Shiny.jpg",
        "Cortex.jpg",
        "Cream.jpg",
        "Fuzzy.jpg",
        "Peach.jpg",
        "Plastic.jpg",
        "Gold.jpg",
    ],
)

check_xray = widgets.Checkbox(value=False, description="XRay")

drag_mode = widgets.Dropdown(
    options=[
        ("contrast", DragMode.CONTRAST),
        ("measurement", DragMode.MEASUREMENT),
        ("pan/zoom", DragMode.PAN),
        ("slicer3D", DragMode.SLICER_3D),
        ("none", DragMode.NONE),
    ],
    value=DragMode.CONTRAST,
    description="Drag mode:",
)

render_mode = widgets.Dropdown(
    options=[
        ("slices", -1.0),
        ("matte", 0.0),
        ("low", 0.3),
        ("medium", 0.6),
        ("high", 1.0),
    ],
    value=1.0,
    description="Render Mode:",
)

mat_caps = widgets.Dropdown(
    options=["Shiny", "Cortex", "Cream", "Fuzzy", "Peach", "Plastic"],
    value="Cortex",
    description="MatCap:",
)

zoom_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=3.0, step=0.1, description="Zoom:"
)
location_label = widgets.Label(value="Location: ")

nv = ipyniivue.NiiVue(
    loading_text="there are no images",
    back_color=[1, 1, 1, 1],
    show_3d_crosshair=True,
    limit_frames_4d=3,
)

nv.opts.multiplanar_show_render = ShowRender.ALWAYS
nv.opts.drag_mode = DragMode.CONTRAST
nv.opts.slice_type = SliceType.MULTIPLANAR
nv.set_clip_plane(0.3, 180, 20)
nv.graph.auto_size_multiplanar = True


@nv.on_canvas_attached
def set_illumination():
    """Set volume render illumination."""
    nv.set_volume_render_illumination(1.0)


def on_xray_change(change):
    """Handle xray change."""
    nv.opts.mesh_xray = 0.05 if change["new"] else 0.0


def on_drag_mode_change(change):
    """Handle drag mode change."""
    nv.opts.drag_mode = change["new"]


def on_render_mode_change(change):
    """Handle render mode change."""
    nv.set_volume_render_illumination(change["new"])


def on_matcap_change(change):
    """Handle matcap change."""
    matcap_name = change["new"]
    matcap_path = f"./matcaps/{matcap_name}.jpg"
    with open(matcap_path, "rb") as f:
        matcap_data = f.read()
    nv.load_mat_cap_texture(matcap_data)


def on_zoom_change(change):
    """Handle zoom change."""
    nv.scene.vol_scale_multiplier = change["new"]


def handle_location_change(data):
    """Handle location change."""
    location_label.value = f"Location: {data.get('string', '')}"


check_xray.observe(on_xray_change, names="value")
drag_mode.observe(on_drag_mode_change, names="value")
render_mode.observe(on_render_mode_change, names="value")
mat_caps.observe(on_matcap_change, names="value")
zoom_slider.observe(on_zoom_change, names="value")
nv.on_location_change(handle_location_change)

on_matcap_change({"new": "Cortex"})

nv.load_volumes(
    [
        {"path": "images/mni152.nii.gz", "cal_min": 30, "cal_max": 80},
        {
            "path": "images/spmMotor.nii.gz",
            "cal_min": 3,
            "cal_max": 8,
            "colormap": "warm",
        },
    ]
)

nv.load_meshes(
    [
        {"path": "images/connectome.jcon"},
        {"path": "images/dpsv.trx", "rgba255": [0, 142, 0, 255]},
    ]
)

display(
    widgets.VBox(
        [
            widgets.HBox([check_xray, drag_mode, render_mode]),
            widgets.HBox([mat_caps, zoom_slider]),
        ]
    )
)
display(nv)
display(location_label)